In [6]:
import  numpy  as  np 
import  pandas  as  pd 
import  string 
from  numpy.random  import  normal, seed 
#from numpy.random import normal,random,seed 
#from scipy.stats import norm 
import  matplotlib.pyplot  as  plt 
plt.style.use ('ggplot') 
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier 
from  sklearn.preprocessing  import  LabelEncoder 
from  sklearn.model_selection  import  GridSearchCV , train_test_split 
#from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split 
from  sklearn.metrics  import  log_loss 
import  xgboost  as  xgb 
from  xgboost.sklearn  import  XGBClassifier ,  XGBRegressor 
import  lightgbm  as  lgb

In [7]:
Data = pd.read_csv('DTTrain.csv')
Data.shape # dataset dimensions
# data content
Data.head()

,time,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,45652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Class_5
1,225646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,Class_3
2,345735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,7.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,Class_5
3,436478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Class_5
4,478129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Class_5


In [10]:
X = Data.drop(['time','target'],axis=1)
y = LabelEncoder().fit_transform(Data.target)
xTrain, xTest, yTrain, yTest = train_test_split(X, y,
                                                test_size=1/3, 
                                                random_state=0)

In [11]:
param = {"objective" : "multi:softprob","eval_metric" : "mlogloss",
         "num_class" : max(y) + 1 }
dtrain = xgb.DMatrix(xTrain, yTrain)

In [15]:
cv_nround = 20
cv_nfold = 3
bst_cv = xgb.cv(param, dtrain, nfold = cv_nfold, 
                num_boost_round=cv_nround,
               verbose_eval=1, show_stdv=False)

[0]	train-mlogloss:1.16171	test-mlogloss:1.18911
[1]	train-mlogloss:0.92361	test-mlogloss:0.96714
[2]	train-mlogloss:0.76343	test-mlogloss:0.82033
[3]	train-mlogloss:0.64553	test-mlogloss:0.71396
[4]	train-mlogloss:0.55661	test-mlogloss:0.63487
[5]	train-mlogloss:0.48713	test-mlogloss:0.57461
[6]	train-mlogloss:0.43178	test-mlogloss:0.52704
[7]	train-mlogloss:0.38724	test-mlogloss:0.48909
[8]	train-mlogloss:0.35004	test-mlogloss:0.45968
[9]	train-mlogloss:0.31938	test-mlogloss:0.43569
[10]	train-mlogloss:0.29544	test-mlogloss:0.41604
[11]	train-mlogloss:0.27257	test-mlogloss:0.40003
[12]	train-mlogloss:0.25386	test-mlogloss:0.38606
[13]	train-mlogloss:0.23777	test-mlogloss:0.37497
[14]	train-mlogloss:0.22328	test-mlogloss:0.36599
[15]	train-mlogloss:0.21177	test-mlogloss:0.35780
[16]	train-mlogloss:0.20191	test-mlogloss:0.35055
[17]	train-mlogloss:0.19215	test-mlogloss:0.34558
[18]	train-mlogloss:0.18286	test-mlogloss:0.34052
[19]	train-mlogloss:0.17623	test-mlogloss:0.33633


In [16]:
bst = xgb.train(param, dtrain,num_boost_round=cv_nround)
xgbPred = bst.predict(xgb.DMatrix(xTest))
xgbPred[:6]

array([[2.1069308e-01, 2.7901703e-02, 1.8191220e-02, 1.0099392e-02,
        7.3311460e-01],
       [8.7904353e-03, 3.8036659e-02, 7.0078514e-04, 9.4865477e-01,
        3.8173276e-03],
       [2.8152226e-02, 9.1238576e-01, 4.8027840e-03, 1.0422038e-02,
        4.4237208e-02],
       [1.9544974e-02, 8.1984884e-01, 3.8502187e-02, 4.4101361e-02,
        7.8002684e-02],
       [2.6142816e-03, 9.7672606e-01, 1.6379076e-03, 1.6507043e-02,
        2.5147011e-03],
       [2.1381283e-03, 3.0357342e-03, 9.8961663e-01, 2.9461491e-03,
        2.2633329e-03]], dtype=float32)

In [17]:
print('XGBoost logloss',log_loss(yTest, xgbPred))

XGBoost logloss 0.32130506982215096
